In [ ]:
import os
os.chdir('drive/My Drive/MLProject_20202')

In [ ]:
from keras import models

In [ ]:
lstm_model = models.load_model('Model/model_25042020.h5')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_json("Data/data_text_embedded.json")
df.head()

,embedded_text,embedded_title
0,"[1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[1, 141, 99, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[1, 205, 206, 207, 208, 209, 210, 211, 212, 21...","[1, 268, 213, 210, 211, 212, 2, 0, 0, 0, 0, 0,..."
2,"[1, 4, 442, 3, 443, 213, 248, 444, 445, 446, 2...","[1, 312, 490, 491, 9047, 2, 0, 0, 0, 0, 0, 0, ..."
3,"[1, 4, 572, 573, 267, 574, 219, 575, 576, 318,...","[1, 272, 103, 573, 572, 2, 0, 0, 0, 0, 0, 0, 0..."
4,"[1, 4, 796, 797, 798, 799, 800, 445, 801, 241,...","[1, 935, 871, 287, 2, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
context_vectors = []
start_sequence = np.array([[1,0,0,0,0,0,0,0]])
feature_extractor = models.Model(inputs = lstm_model.inputs, outputs = lstm_model.get_layer(index=6).output)
for i in df.index:
    vector_to_context = np.reshape(np.array(df.embedded_text[i]), (-1, len(df.embedded_text[0])))
    features = feature_extractor([vector_to_context, start_sequence])
    context_vectors.append(features)

In [ ]:
import tensorflow as tf

In [ ]:
df_cat = pd.read_csv('Data/data_all_categorical.csv', encoding='utf-8')
df_cat.head()

,telecommuting,has_company_logo,has_questions,fraudulent,employment_type_Contract,employment_type_Full-time,employment_type_Other,employment_type_Part-time,employment_type_Temporary,required_experience_Associate,required_experience_Director,required_experience_Entry level,required_experience_Executive,required_experience_Internship,required_experience_Mid-Senior level,required_experience_Other
0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0


In [ ]:
y = df_cat.fraudulent

In [ ]:
df_cat_no_target = df_cat.loc[:, df_cat.columns != 'fraudulent']
df_cat_no_target.head()

,telecommuting,has_company_logo,has_questions,employment_type_Contract,employment_type_Full-time,employment_type_Other,employment_type_Part-time,employment_type_Temporary,required_experience_Associate,required_experience_Director,required_experience_Entry level,required_experience_Executive,required_experience_Internship,required_experience_Mid-Senior level,required_experience_Other
0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1
3,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
4,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0


In [ ]:
X = []
for i in df.index:    
    X.append(tf.concat([tf.reshape(context_vectors[i], [-1]), np.array(df_cat_no_target.iloc[i])], axis=0))

In [ ]:
np.array(X)

array([[0.7100586 , 0.72235614, 0.4698896 , ..., 1.        , 0.        ,
        0.        ],
       [0.7100586 , 0.72235614, 0.4698896 , ..., 0.        , 0.        ,
        1.        ],
       [0.7100586 , 0.72235614, 0.4698896 , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.7100586 , 0.72235614, 0.4698896 , ..., 0.        , 0.        ,
        1.        ],
       [0.7100586 , 0.72235614, 0.4698896 , ..., 0.        , 0.        ,
        1.        ],
       [0.7100586 , 0.72235614, 0.4698896 , ..., 0.        , 1.        ,
        0.        ]], dtype=float32)

In [ ]:
X[0].shape

TensorShape([271])

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic_reg = LogisticRegression(random_state=0).fit(X,y)

In [ ]:
logistic_reg.predict([X[5]])

array([0])

In [ ]:
logistic_reg.score(X, y)

0.9515659955257271